In [2]:
import pandas as pd
import numpy as np

In [3]:
db = pd.read_csv('temperature-quotidienne-departementale.csv', sep=';')

db.dtypes

date_obs                   object
code_insee_departement     object
departement                object
tmin                      float64
tmax                      float64
tmoy                      float64
dtype: object

In [4]:




# db['date_obs'] = pd.to_datetime(db['date_obs'], format='%d/%m/%Y')
db['date_obs'] = pd.to_datetime(db['date_obs'], format='%Y-%m-%d')

print(db)

db.sort_values(by=['code_insee_departement','date_obs'], inplace=True)


         date_obs code_insee_departement     departement   tmin   tmax   tmoy
0      2019-10-22                     21       Côte-d'Or  10.90  16.00  13.45
1      2019-10-22                     73          Savoie  11.70  18.65  15.18
2      2019-10-22                     29       Finistère   7.77  15.48  11.63
3      2019-10-22                     48          Lozère   9.70  16.50  13.10
4      2019-10-22                     49  Maine-et-Loire   4.40  15.10   9.75
...           ...                    ...             ...    ...    ...    ...
178267 2023-01-31                     68       Haut-Rhin  -2.15   6.75   2.30
178268 2023-01-31                     49  Maine-et-Loire   3.40   9.20   6.30
178269 2023-01-31                     52     Haute-Marne   2.10   4.95   3.52
178270 2023-01-31                     57         Moselle  -0.80   5.30   2.25
178271 2023-01-31                     63     Puy-de-Dôme   0.70   8.70   4.70

[178272 rows x 6 columns]


In [5]:
# convert date_obs_obs to int starting from 0
db['date_obs'] = db['date_obs'] - db['date_obs'].min()

db['date_obs'] = db['date_obs'].dt.days
print(db)

        date_obs code_insee_departement departement   tmin   tmax   tmoy
143434         0                     01         Ain   5.40   9.40   7.40
143554         1                     01         Ain   4.60  14.10   9.35
137985         2                     01         Ain   7.20  14.60  10.90
146099         3                     01         Ain   6.50  14.20  10.35
154634         4                     01         Ain  11.00  14.40  12.70
...          ...                    ...         ...    ...    ...    ...
176392      1852                     95  Val-d'Oise   3.13   4.77   3.95
178202      1853                     95  Val-d'Oise   1.23   3.33   2.28
176563      1854                     95  Val-d'Oise  -4.10   4.77   0.33
176484      1855                     95  Val-d'Oise   3.40   9.53   6.47
176517      1856                     95  Val-d'Oise  -0.90   9.47   4.28

[178272 rows x 6 columns]


In [12]:
# extract a single line of index 5
line = db.iloc[5]
line2 = db.iloc[700]
print(line)
print(line2)

date_obs                     5
code_insee_departement      01
departement                Ain
tmin                       6.1
tmax                      15.7
tmoy                      10.9
Name: 142230, dtype: object
date_obs                  700
code_insee_departement     01
departement               Ain
tmin                      2.4
tmax                      5.4
tmoy                      3.9
Name: 806, dtype: object


In [18]:
def interpolation(X1,X2,Y1,Y2,XInconnu):
    #prevent division by 0
    if(X2 == X1):
        return Y1
    YInconnu = Y2*((XInconnu-X1)/(X2-X1))+Y1*((X2-XInconnu)/(X2-X1))
    return YInconnu



def interpol_line(line1:pd.core.series.Series, line2:pd.core.series.Series, col2:str, target:int, toInterpol:list[str]=[], toNotInterpol:list[str]=[])->pd.core.series.Series:
    #col2 : colonne de temps 
    #target : temps de la nouvelle ligne
    #les temps doivent être des int ou float, peu importe l'échelle

    x1 = line1[col2]
    x2 = line2[col2]
    res = line1.copy()
    for col in res.index:
        res[col] = np.nan 
    res[col2] = target
    # interpolate all ints in the lines except col2
    for col in line1.index:
        if(col != col2  ):
            #check that col is numeric
            if(pd.api.types.is_numeric_dtype(line1[col])):
                if(col in toInterpol or col not in toNotInterpol):
                    y1 = line1[col]
                    y2 = line2[col]
                    print("interpolating", col, "from", y1, "to", y2, "for", target, "between", x1, "and", x2)
                    res[col]=(interpolation(x1, x2, y1, y2, target))
                    print("result", res[col])
    return res

(interpol_line(line, line2, 'date_obs', 450))


interpolating tmin from 6.1 to 2.4 for 450 between 5 and 700
result 3.730935251798561
interpolating tmax from 15.7 to 5.4 for 450 between 5 and 700
result 9.105035971223021
interpolating tmoy from 10.9 to 3.9 for 450 between 5 and 700
result 6.417985611510792


date_obs                       450
code_insee_departement         NaN
departement                    NaN
tmin                      3.730935
tmax                      9.105036
tmoy                      6.417986
Name: 142230, dtype: object

In [17]:
db.replace('2A', 102, inplace=True)
db.replace('2B', 202, inplace=True)


In [9]:
#show stats
print(db.describe())

            date_obs           tmin           tmax           tmoy
count  178272.000000  178272.000000  178272.000000  178272.000000
mean      928.000000       8.406814      17.760515      13.083663
std       536.071151       6.192130       7.984985       6.749619
min         0.000000     -16.800000      -7.520000     -10.300000
25%       464.000000       3.900000      11.800000       8.080000
50%       928.000000       8.500000      17.350000      12.750000
75%      1392.000000      13.150000      23.730000      18.280000
max      1856.000000      29.400000      42.770000      33.800000
